In [6]:
# set precision
pd.set_option('display.precision', 14)
# import modules
import pandas as pd

# import Co training data using the pandas library and convert them into a dataframe
raw_data = pd.read_excel('data/Co_temperature.xlsx')
raw_data.dropna(axis='rows', inplace=True)

random_state = 1
raw_data.describe()



,Unnamed: 0,Oxygen vacancy,Polarization,Charge,Electronegativity,Radius A-site,Radius B-site,Tolerance factor,Temperature
count,2895.00000000000000,2895.00000000000000,2895.00000000000000,2895.00000000000000,2895.00000000000000,2895.00000000000000,2895.00000000000000,2895.00000000000000,2895.00000000000000
mean,1586.20863557858365,0.25374497011683,7.96945326090710,2.62581894972079,1.24828639732649,1.40878574285915,0.62058314223614,0.99145686208105,491.14853195164073
std,879.48421618489101,0.14855562838597,5.13948052323340,0.35940128546514,0.13197701780728,0.06894585859586,0.02381051198844,0.02500012220108,210.92017615391325
min,0.00000000000000,-0.00229500000000,2.72371750632620,2.00000000000000,0.96478260869565,1.24020000000000,0.59600000000000,0.92689034414490,25.00000000000000
25%,843.50000000000000,0.14858250000000,4.99301561365949,2.50000000000000,1.14000000000000,1.37600000000000,0.61000000000000,0.97616997689958,350.00000000000000
50%,1594.00000000000000,0.23966500000000,5.69126378286684,2.50000000000000,1.19600000000000,1.39750000000000,0.61082191780822,0.99158127854083,500.00000000000000
75%,2337.50000000000000,0.34171764200086,8.92058622914923,2.71428571428571,1.35714285714286,1.44000000000000,0.61875000000000,1.00473133232742,650.00000000000000
max,3106.00000000000000,0.98181800000000,32.82656162883060,4.23677581863980,1.69536585365854,1.61000000000000,0.74500000000000,1.06426785943894,800.00000000000000


In [7]:
from sklearn.utils import shuffle
# take the oxygen vacancy as the label
Y = raw_data.loc[:, 'Oxygen vacancy'].values

# take the other collumns as the features
X = raw_data.iloc[:, 3:]


X, y = shuffle(X, Y, random_state=random_state)
X.head()

,Polarization,Charge,Electronegativity,Radius A-site,Radius B-site,Tolerance factor,Temperature
2417,10.88061039101431,2.58974358974359,1.42051282051282,1.4400,0.602000,1.01223318831030,700
1543,5.01792114695341,2.50000000000000,1.20500000000000,1.3400,0.610000,0.97181855055382,500
1600,5.08241496339231,2.22727272727273,1.11590909090909,1.4400,0.634375,0.99563499927754,500
238,4.87126828290924,2.50000000000000,1.13500000000000,1.4250,0.610000,1.00264115383631,25
2824,9.39222762639121,2.74358974358974,1.37641025641026,1.3945,0.611750,0.99069219580101,750


In [8]:
# read Fe training data
data_fe = pd.read_excel('data/Fe_temperature.xlsx')
data_fe.dropna(axis='rows', inplace=True)
Y_fe = data_fe.loc[:, 'Oxygen vacancy'].values

# take the other collumns as the features and do shuffle
X_fe = data_fe.iloc[:, 3:]

X_fe, y_fe = shuffle(X_fe, Y_fe, random_state=random_state)
X_fe.head()

,Polarization,Charge,Electronegativity,Radius A-site,Radius B-site,Tolerance factor,Temperature
966,2.69113080014615,2.05000000000000,0.91050000000000,1.5975,0.6450,1.04641116863249,400
860,3.81557427418059,2.00000000000000,1.26333333333333,1.6100,0.6620,1.04194056168847,400
2006,2.77777777777778,2.00000000000000,1.03000000000000,1.4400,0.6450,0.99030571873985,700
1409,6.07638888888889,2.27272727272727,1.10636363636364,1.4400,0.6445,0.99055522887307,550
1654,5.05902991797449,2.50000000000000,1.14500000000000,1.3945,0.6450,0.97409747765976,600


In [ ]:
# Hide warnings
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
# Scikit-learn packages
# from sklearn.ensemble import ExtraTreesRegressor
from lightgbm import LGBMRegressor
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

reg = LGBMRegressor(n_estimators=100, random_state=random_state)
reg.fit(X_train, y_train)
print(f"Test R2 score: {reg.score(X_test, y_test):f}")
predictions = reg.predict(X_test)



In [ ]:
from sklearn.inspection import partial_dependence
from sklearn.inspection import plot_partial_dependence
from time import time
from sklearn.pipeline import make_pipeline
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
est = make_pipeline(LGBMRegressor(random_state=random_state))

est.fit(X_train, y_train)
features = ["Polarization", "Charge", "Electronegativity", "Radius A-site", "Radius B-site", "Temperature"]
print("Computing partial dependence plots...")

tic = time()
plot_partial_dependence(est, X_train, features,
                        n_jobs=3, grid_resolution=20)
print("done in {:.3f}s".format(time() - tic))
fig = plt.gcf()
fig.suptitle('Partial dependence of Co-data with LGBMRegressor')
fig.subplots_adjust(hspace=0.3)

# display = PartialDependenceDisplay.from_estimator(
#     est,
#     X,
#     features,
#     kind="both",
#     subsample=50,
#     n_jobs=3,
#     grid_resolution=20,
#     random_state=0,
#     ice_lines_kw={"color": "tab:blue", "alpha": 0.2, "linewidth": 0.5},
#     pd_line_kw={"color": "tab:orange", "linestyle": "--"},
# )
# print(f"done in {time() - tic:.3f}s")
# display.figure_.suptitle(
#     "Partial dependence with Gradient Boosting"
# )
# display.figure_.subplots_adjust(wspace=0.4, hspace=0.3)
